In [1]:
!pip install transformers llama-cpp-python portalocker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 MB 14.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 6.3 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.75-cp310-cp310-linux_x86_64.whl size=3688344 sha256=7fa5da74acdc4ae86a25db0b792d91ebc08e00c71a042b4e8bb4bb586222bc71
  Stored in directory: /root/.cache/pip/wheels/5e/df/9a/e4bb2e48bfa64fb174f0f786296c8507dbebea2a112f1adf8d
Successfully built llama-cpp-python


In [2]:
from huggingface_hub import HfApi, hf_hub_download

# Define the repository and the model identifier
repo_id = "TheBloke/phi-2-GGUF"
model_filename = "phi-2.Q4_K_M.gguf"

# Initialize the HfApi object
api = HfApi()

# Download the model file to a specified local directory
local_model_path = hf_hub_download(repo_id=repo_id, filename=model_filename, cache_dir="./")

print(f"Model downloaded to: {local_model_path}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


phi-2.Q4_K_M.gguf:   0%|          | 0.00/1.79G [00:00<?, ?B/s]

Model downloaded to: ./models--TheBloke--phi-2-GGUF/snapshots/5a454d977c6438bb9fb2df233c8ca70f21c87420/phi-2.Q4_K_M.gguf


In [22]:
import pandas as pd

# Example: Load your data here
# For the purpose of this example, let's assume a simple DataFrame with columns 'text' and 'label'
url = "https://raw.githubusercontent.com/clairett/pytorch-sentiment-classification/master/data/SST2/train.tsv"
data = pd.read_csv(url, delimiter='\t', header=None)
texts = data[0].tolist()
labels = data[1].tolist()

from sklearn.model_selection import train_test_split


# Split the data into training and test sets
# Here, 80% of the data will be used for training and 20% for testing
texts_train, texts_test, labels_train, labels_test = train_test_split(texts, labels, test_size=0.99, random_state=42)



In [7]:
print(len(texts_train))

346


In [23]:

from llama_cpp import Llama


llm = Llama(
    model_path=local_model_path,
    n_gpu_layers=10,  # Uncomment to use GPU acceleration
    seed=1337,  # Uncomment to set a specific seed
    n_ctx=1000,  # Uncomment to increase the context window
    verbose=False,  # Uncomment to enable verbose mode
    embedding=True,
)

In [24]:
from tqdm import tqdm
import numpy as np

# Initialize an empty list to hold the result
train_utts, test_utts = ["Weather is nice too", "she is a terrible person"], ["what a terrible situation"]
train_embeddings, test_embeddings = [], []

print("Extracting embeddings for training data")
for utt in tqdm(texts_train):
      embed = np.array(llm.create_embedding(utt)['data'][0]['embedding'])

      # raise exception if embed is nan
      if np.isnan(embed).any():
            raise ValueError('Embedding is nan')

      train_embeddings.append(embed)

# print("Extracting embeddings for test data")
# for utt in tqdm(texts_test):
#       embed = np.array(llm.create_embedding(utt)['data'][0]['embedding'])

#       # raise exception if embed is nan
#       if np.isnan(embed).any():
#             raise ValueError('Embedding is nan')

#       test_embeddings.append(embed)




Extracting embeddings for training data


100%|██████████| 69/69 [04:35<00:00,  4.00s/it]


In [32]:

embed = []

for e in train_embeddings:
  embed.append(np.mean(e, axis=0))

